<a href="https://colab.research.google.com/github/Guhan2348519/LLM-lab-tasks/blob/main/2348519_LLM_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import torch
from transformers import BertForQuestionAnswering, BertTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup
import numpy
import torch

In [2]:
import json

# Load the JSON file
with open("squad_dataset.json", "r") as file:
    dataset = json.load(file)



In [4]:
print(json.dumps(dataset, indent=4))

{
    "features": [
        {
            "feature_idx": 0,
            "name": "question",
            "type": {
                "dtype": "string",
                "_type": "Value"
            }
        },
        {
            "feature_idx": 1,
            "name": "answer",
            "type": {
                "dtype": "string",
                "_type": "Value"
            }
        }
    ],
    "rows": [
        {
            "row_idx": 0,
            "row": {
                "question": "To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?",
                "answer": "Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend \"Venite Ad Me Omnes\". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of 

In [10]:
import json
from transformers import BertTokenizer

def preprocess_json_data(json_data):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    examples = []
    for row in json_data["rows"]:
        question = row["row"]["question"]
        answer = row["row"]["answer"]

        # Tokenization
        tokenized_inputs = tokenizer(question, answer, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
        input_ids = tokenized_inputs['input_ids'].squeeze(0)
        attention_mask = tokenized_inputs['attention_mask'].squeeze(0)

        examples.append({
            'input_ids': input_ids,
            'attention_mask': attention_mask
        })
    return examples

# Load JSON data
with open('squad_dataset.json', 'r') as f:
    json_data = json.load(f)

# Preprocess the JSON data
preprocessed_examples = preprocess_json_data(json_data)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [12]:
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from transformers import BertForQuestionAnswering, BertTokenizer, AdamW, get_linear_schedule_with_warmup

# Load pre-trained BERT model
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader))

# Fine-tuning loop
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids, attention_mask, start_positions, end_positions = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

# Save the fine-tuned model
model.save_pretrained('fine_tuned_bert_model')


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


NameError: name 'train_dataloader' is not defined

In [2]:
import json
import torch
from transformers import BertForQuestionAnswering, BertTokenizer
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

def preprocess_json_data(json_data):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    examples = []
    for paragraph in json_data["rows"]:
        context = paragraph["row"]["answer"]
        question = paragraph["row"]["question"]

        # Tokenization
        tokenized_inputs = tokenizer(question, context, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
        input_ids = tokenized_inputs['input_ids'].squeeze(0)
        attention_mask = tokenized_inputs['attention_mask'].squeeze(0)

        examples.append({
            'input_ids': input_ids,
            'attention_mask': attention_mask
        })
    return examples

# Load JSON data
with open('squad_dataset.json', 'r') as f:
    json_data = json.load(f)

# Preprocess the JSON data
preprocessed_examples = preprocess_json_data(json_data)

# Load pre-trained BERT model
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = 1000  # You should set this according to your dataset size and batch size
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Fine-tuning loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for example in preprocessed_examples:
        input_ids, attention_mask = example['input_ids'], example['attention_mask']
        # Assuming you have start_positions and end_positions available
        start_positions = torch.tensor([0])  # Change 0 to actual start position
        end_positions = torch.tensor([10])  # Change 10 to actual end position
        outputs = model(input_ids=input_ids.unsqueeze(0), attention_mask=attention_mask.unsqueeze(0), start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

# Save the fine-tuned model
model.save_pretrained('fine_tuned_bert_model')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
